# Использование модели метчинга для поиска топ-5 услуг для новых услуг клиник

In [1]:
import pandas as pd
import numpy as np

from matcher.matcher import SentenceSimilarity
from configs.config_matcher import MatcherConfig
from utils.compute_metrics import mark, mapk, f1, get_pivot_table
from tqdm import tqdm

tqdm.pandas()

/home/evgen/desktop/ranker/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-19 12:22:16.886777: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 12:22:16.895659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 12:22:16.904701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has 

Для проверки работы алгоритма, будет использоваться датасет из которого удалены дубликаты в столбце `local_name`

In [2]:
data = pd.read_csv("data/data_for_matcher.csv")

In [3]:
data.sample(5)

,local_name,local_id,parent_id,type,gt_type_name,parent_id_name,site_active,service_name,lpu_name
1209,Определение фракции трансферрина (CDT) (диагно...,5728.0,64.0,laboratory_tests,Биохимия,Лабораторная диагностика,True,Определение фракции трансферрина (CDT) (диагно...,ДНКОМ
15,Волчаночный антикоагулянт,5540.0,64.0,laboratory_tests,Биохимия,Лабораторная диагностика,True,Волчаночный антикоагулянт (Lupus anticoagulant...,Инвитро
154,Фруктозамин,5594.0,64.0,laboratory_tests,Биохимия,Лабораторная диагностика,True,Фруктозамин (Fructosamine),Инвитро
212,Миоглобин,5618.0,64.0,laboratory_tests,Биохимия,Лабораторная диагностика,True,Миоглобин (Myoglobin),Инвитро
93,HE4,6117.0,64.0,laboratory_tests,Биохимия,Лабораторная диагностика,True,Опухолевый маркёр HE4 (Human epididymis protei...,Инвитро


Параметры для конфига указаны в файле `configs/config_matcher.py`

In [4]:
config = MatcherConfig()
model = SentenceSimilarity(config=config)

In [5]:
predict = data['service_name'].progress_apply(lambda x: model.get_top_k(x, top_k=5))

  0%|          | 0/4047 [00:00<?, ?it/s]/home/evgen/desktop/ranker/matcher/matcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.embedder.encode(sentences=texts, convert_to_tensor=True)).clone().detach()
100%|██████████| 4047/4047 [00:29<00:00, 136.23it/s]


In [6]:
predict_list = predict.apply(lambda x: [key['local_name'] for key in x]).values.tolist()
predict_list[:2]

[['Выявление стресса и его стадии',
  'Комплексная оценка оксидативного стресса (7 параметров)',
  'Комплексное исследование, ОСМ',
  'Обследование домашнего персонала (расширенное)',
  'Комплексное исследование на гормоны (12 показателей)'],
 ['Клинический анализ крови: общий анализ, лейкоцитарная формула, СОЭ (с обязательной микроскопией мазка крови)',
  'Клинический анализ крови: общий анализ, лейкоцитарная формула, СОЭ (с микроскопией мазка крови при выявлении патологических изменений)',
  'Лейкоцитарная формула (с обязательной микроскопией мазка крови)',
  'Лейкоцитарная формула (с микроскопией мазка крови при выявлении патологических изменений)',
  'Клинический анализ крови с микроскопией лейкоцитарной формулы']]

In [7]:
actual_list = data['local_name'].apply(lambda x: [x]).values.tolist()
actual_list[:5]

[['Выявление стресса и его стадии'],
 ['Клинический анализ крови с микроскопией лейкоцитарной формулы'],
 ['Лейкоцитарная формула (с обязательной микроскопией мазка крови)'],
 ['Клинический анализ крови (c лейкоцитарной формулой)'],
 ['Клинический анализ крови: общий анализ, лейкоцитарная формула, СОЭ (с микроскопией мазка крови при выявлении патологических изменений)']]

In [8]:
get_pivot_table(actual_list, predict_list, top_k=5)

,k,mark,mapk,f1_score
0,1,0.857672,0.857672,0.857672
1,2,0.888312,0.888312,0.888312
2,3,0.894572,0.894572,0.894572
3,4,0.897475,0.897475,0.897475
4,5,0.898958,0.898958,0.898958


In [9]:
model.get_top_k("Антитела класса IgG к вирусу Varicella-Zoster", top_k=5)

/home/evgen/desktop/ranker/matcher/matcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.embedder.encode(sentences=texts, convert_to_tensor=True)).clone().detach()


[{'local_name': 'Varicella Zoster Virus, IgG', 'score': 0.8591366},
 {'local_name': 'Varicella Zoster Virus, IgA', 'score': 0.7878473},
 {'local_name': 'Varicella Zoster Virus, IgM', 'score': 0.771694},
 {'local_name': 'Антитела к нуклеосомам класса IgG', 'score': 0.76468045},
 {'local_name': 'Антитела к Poliomyelitis virus, IgG', 'score': 0.70515966}]

In [10]:
model.get_top_k("Кровь", top_k=5)

[{'local_name': 'Взятие крови из пальца', 'score': 0.6224702},
 {'local_name': 'Панель X-Registar кровь', 'score': 0.6061321},
 {'local_name': 'Взятие венозной крови', 'score': 0.6008311},
 {'local_name': 'Время свертывания крови и длительность кровотечения',
  'score': 0.5856257},
 {'local_name': 'Взятие крови из пятки', 'score': 0.5807512}]

In [11]:
model.get_top_k("Анализ крови", top_k=5)

[{'local_name': 'Анализ крови на органические кислоты', 'score': 0.74609447},
 {'local_name': 'Анализ крови на аминокислоты (13 показателей)',
  'score': 0.7190759},
 {'local_name': 'Клинический анализ крови (c лейкоцитарной формулой)',
  'score': 0.7185571},
 {'local_name': 'Анализ крови на ацилкарнитины (взрослые)',
  'score': 0.71110886},
 {'local_name': 'Клинический и биохимический анализы крови - основные показатели',
  'score': 0.7096769}]